# Proceso de ETL con los datos disponibles

### Extracción (Extract)

In [2]:
import pandas as pd

# Cargar los archivos CSV en DataFrames
df_australian_user_reviews = pd.read_csv('Datos-Csv/australian_users_reviews.csv')
df_australian_users_items = pd.read_csv('Datos-Csv/australian_users_items.csv')
df_output_steam_games = pd.read_csv('Datos-Csv/output_steam_games.csv')


C:\Users\Facundete\AppData\Local\Temp\ipykernel_3032\640466342.py:6: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_output_steam_games = pd.read_csv('Datos-Csv/output_steam_games.csv')


##### Validacion 

In [ ]:
df_australian_user_reviews.head()

In [3]:
df_australian_users_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [ ]:
df_output_steam_games.head()

### Transformación (Transform)

##### Funciones de normalizacion

In [4]:
import pandas as pd

#normalizacion en general
def normalize_dataframe(df, null_tolerance, column_name=None, min_chars=None):
    # Eliminar duplicados
    df.drop_duplicates(inplace=True)

    # Eliminar filas con más de 'null_tolerance' valores nulos
    df.dropna(thresh=null_tolerance, inplace=True)

    # Eliminar filas que contienen listas vacías
    for col in df.columns:
        df = df[df[col].apply(lambda x: x != [] if isinstance(x, list) else True)]

    # Si se proporciona un nombre de columna y un mínimo de caracteres,
    # eliminar las filas donde el contenido de esa columna tiene menos caracteres que 'min_chars'
    if column_name and min_chars:
        df = df[df[column_name].str.len() >= min_chars]

    # Si se proporciona un nombre de columna,
    # eliminar las filas donde el contenido de esa columna es una lista vacía
    if column_name:
        df = df[df[column_name].apply(lambda x: x != [] if isinstance(x, list) else True)]

    return df


In [5]:
import pandas as pd

#normalizacion de fechas
def normalize_date_column(df, date_column):
    # Intentamos convertir la columna de fechas al formato deseado
    try:
        df[date_column] = pd.to_datetime(df[date_column], format='%Y-%m-%d')
    except ValueError:
        # Si la conversión falla, eliminamos las filas con fechas en un formato incorrecto
        df = df[pd.to_datetime(df[date_column], errors='coerce').notna()]
    return df


##### output_steam_games_normalizado

In [ ]:
df_output_steam_games_normalizado=normalize_dataframe(df_output_steam_games,5)
df_output_steam_games_normalizado=normalize_date_column(df_output_steam_games,'release_date')

In [ ]:
df_output_steam_games_normalizado.to_csv("Datos-Normalizados/output_steam_games_normalizado.csv")

##### australian_user_items_normalizado

In [6]:
df_australian_users_items_normalizado=normalize_dataframe(df_australian_users_items,0,'items',4)

In [10]:
df_australian_users_items_normalizado

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88303,76561198320136420,321,76561198320136420,http://steamcommunity.com/profiles/76561198320...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88304,ArkPlays7,4,76561198321542621,http://steamcommunity.com/id/ArkPlays7,"[{'item_id': '218230', 'item_name': 'PlanetSid..."
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."


In [8]:
df_australian_users_items_normalizado.to_csv('Datos-Normalizados/australian_users_items_normalizado.csv')

##### australian_user_reviews_normalizado

Análisis de las emociones de las reseñas

In [ ]:
import pandas as pd
from textblob import TextBlob

# Asegurarse de que 'reviews' sea una lista de diccionarios
df_australian_user_reviews['reviews'] = df_australian_user_reviews['reviews'].apply(lambda x: eval(x) if pd.notna(x) else [])

# Extraer el contenido de la columna 'review'
df_australian_user_reviews['review_content'] = df_australian_user_reviews['reviews'].apply(lambda reviews: [review['review'] for review in reviews])

# Función para realizar el análisis de sentimiento y devolver el puntaje
def get_sentiment_score(review):
    analysis = TextBlob(str(review))
    if analysis.sentiment.polarity > 0:
        return 2  # Positivo
    elif analysis.sentiment.polarity == 0:
        return 1  # Neutral
    else:
        return 0  # Negativo

# Aplicar el análisis de sentimiento y reemplazar el texto en 'review' con el puntaje
df_australian_user_reviews['reviews'] = df_australian_user_reviews['reviews'].apply(lambda reviews: [{'sentiment_analysis': get_sentiment_score(review['review']), **review} for review in reviews])

# Imprimir las primeras filas del DataFrame resultante
print(df_australian_user_reviews[['reviews']])


In [29]:
df_australian_user_reviews[['reviews']]

0        [{'sentiment_analysis': 2, 'funny': '', 'poste...
1        [{'sentiment_analysis': 2, 'funny': '', 'poste...
2        [{'sentiment_analysis': 2, 'funny': '', 'poste...
3        [{'sentiment_analysis': 2, 'funny': '', 'poste...
4        [{'sentiment_analysis': 1, 'funny': '3 people ...
                               ...                        
25794    [{'sentiment_analysis': 2, 'funny': '', 'poste...
25795    [{'sentiment_analysis': 1, 'funny': '', 'poste...
25796    [{'sentiment_analysis': 1, 'funny': '1 person ...
25797    [{'sentiment_analysis': 2, 'funny': '', 'poste...
25798    [{'sentiment_analysis': 2, 'funny': '1 person ...
Name: reviews, Length: 25799, dtype: object

In [27]:
df_australian_user_reviews.to_csv('Datos-Normalizados/australian_user_reviews_normalizado.csv')

### Carga (Load)